In [2]:
pip install pandas sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [4]:
# ingest_owid_to_postgres.py
import pandas as pd
from sqlalchemy import create_engine

# 1) OWID "latest" CSV endpoints (locked)
URL_CASES_DEATHS = "https://catalog.ourworldindata.org/garden/covid/latest/cases_deaths/cases_deaths.csv"
URL_VAX = "https://catalog.ourworldindata.org/garden/covid/latest/vaccinations_global/vaccinations_global.csv"
URL_POLICY = "https://catalog.ourworldindata.org/garden/covid/latest/oxcgrt_policy/oxcgrt_policy.csv"

# 2) Postgres connection
engine = create_engine(
    "postgresql+psycopg2://zoelin@localhost:5432/covid_owid"
)

def load_csv_to_table(url: str, schema: str, table: str):
    df = pd.read_csv(url)
    # Standardize common keys if present
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df.to_sql(table, engine, schema=schema, if_exists="replace", index=False)
    print(f"Loaded {schema}.{table}: {len(df):,} rows, {len(df.columns)} cols")

if __name__ == "__main__":
    # Create schemas if they don't exist (simple SQL)
    with engine.begin() as conn:
        conn.exec_driver_sql("CREATE SCHEMA IF NOT EXISTS raw;")
        conn.exec_driver_sql("CREATE SCHEMA IF NOT EXISTS mart;")

    load_csv_to_table(URL_CASES_DEATHS, "raw", "cases_deaths")
    load_csv_to_table(URL_VAX, "raw", "vaccinations")
    load_csv_to_table(URL_POLICY, "raw", "gov_response")

Loaded raw.cases_deaths: 535,599 rows, 40 cols
Loaded raw.vaccinations: 203,057 rows, 29 cols
Loaded raw.gov_response: 202,760 rows, 31 cols
